# Importing needed packages

In [1]:
import re
import unicodedata
import pprint

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
pp = pprint.PrettyPrinter(indent=1, width=140, compact=True)

# Parsing main page

## Getting the code for the page with movie

In [3]:
# f = requests.get("https://www.kinopoisk.ru/film/339/")

## Getting the code from the downloaded page

In [4]:
path_front = "./data/indiana_front.html"
with open(path_front, 'r', encoding="utf-8") as f:
    indiana_front = f.read()

Passing the code into the `BeautifulSoup` constructor

In [5]:
soup_front = BeautifulSoup(indiana_front, "html.parser")

In [6]:
movie_dict = {}

## Parsing movie's id

In [7]:
movie_id_dict = {}

In [8]:
film_id = soup_front.find_all("link", attrs={"href": re.compile("https://www.kinopoisk.ru/film/")})[0]

In [9]:
film_id["href"].split("/")[-2]

'339'

In [10]:
movie_id_dict["id"] = int(film_id["href"].split("/")[-2])

## Parsing movie's name

In [11]:
movie_name_dict = {}

In [12]:
movie_name = soup_front.find_all("h1", attrs={"class": re.compile("styles_title")})[0]

In [13]:
movie_name.get_text()

'Индиана Джонс: В поисках утраченного ковчега (1981)'

In [14]:
movie_name_original = soup_front.find_all(
    "span", attrs={"class": re.compile("styles_originalTitle")}
)[0]

In [15]:
movie_name_original.get_text()

'Raiders of the Lost Ark'

In [16]:
movie_name_dict["Название"] = movie_name.get_text()
movie_name_dict["Оргинальное название"] = movie_name_original.get_text()

## Parsing information about the movie

In [17]:
movie_info_dict = {}

In [18]:
movie_info_divs = soup_front.find_all(attrs={"data-test-id": "encyclopedic-table"})[0]

In [19]:
values = []
for div in movie_info_divs.find_all(
    "div", attrs={"class": re.compile("styles_value"), "data-tid": re.compile(".*")}
):
    current_value = div.get_text().replace("сборы", "")
    current_value = unicodedata.normalize("NFKD", current_value)
    if "слова" not in current_value:
        values.append(current_value)

In [20]:
titles = []
for div in movie_info_divs.find_all(class_=re.compile("styles_title")):
    current_title = div.get_text()
    titles.append(current_title)

In [21]:
assert len(values) == len(titles)
movie_info_dict = dict(zip(titles, values))
movie_dict = {**movie_name_dict, **movie_info_dict}

In [22]:
movie_dict

{'Название': 'Индиана Джонс: В поисках утраченного ковчега (1981)',
 'Оргинальное название': 'Raiders of the Lost Ark',
 'Год производства': '1981',
 'Страна': 'США',
 'Жанр': 'приключения, боевик, комедия',
 'Слоган': '«Indiana Jones - the new hero from the creators of JAWS and STAR WARS»',
 'Режиссер': 'Стивен Спилберг',
 'Сценарий': 'Лоуренс Кэздан, Джордж Лукас, Филип Кауфман',
 'Продюсер': 'Ховард Дж. Казанян, Джордж Лукас, Фрэнк Маршалл, ...',
 'Оператор': 'Дуглас Слоком',
 'Композитор': 'Джон Уильямс',
 'Художник': 'Норман Рейнольдс, Айвор Лесли Дилли, Дебора Нэдулмэн, ...',
 'Монтаж': 'Майкл Кан, Джордж Лукас',
 'Бюджет': '$18 000 000',
 'Сборы в США': '$212 222 025',
 'Сборы в мире': '+ $141 766 000 = $353 988 025',
 'Зрители': '88.1 млн , 6.4 млн , 4.2 млн , ...',
 'Премьера в мире': '12 июня 1981, ...',
 'Релиз на DVD': '20 ноября 2003, «Premier Digital»',
 'Возраст': '12+',
 'Рейтинг MPAA': 'PGрекомендуется присутствие родителей',
 'Время': '115 мин. / 01:55'}

## Parsing movie's rating

In [23]:
movie_rating_dict = {}

In [24]:
movie_rating_kinopoisk = soup_front.find_all("a", attrs={"class": re.compile("film-rating-value")})[0]

In [25]:
movie_rating_kinopoisk.get_text()

'8.0'

In [26]:
movie_rating_count_kinopoisk = soup_front.find_all("span", attrs={"class": re.compile("styles_count")})[0]

In [27]:
movie_rating_count_kinopoisk.get_text()

'142 538 оценок'

In [28]:
movie_rating_imdb = soup_front.find_all(
    "span", attrs={"class": re.compile("styles_valueSection")}
)[0]

In [29]:
movie_rating_imdb.get_text()

'IMDb: 8.40'

In [30]:
movie_rating_count_imdb = soup_front.find_all(
    "div", attrs={"class": re.compile("film-sub-rating")}
)[0].find(
    "span", attrs={"class": re.compile("styles_count")}
)

In [31]:
movie_rating_count_imdb.get_text()

'938 581 оценка'

In [32]:
movie_rating_dict["Рейтинг кинопоиска"] = movie_rating_kinopoisk.get_text()
movie_rating_dict["Количество оценок кинопоиска"] = movie_rating_count_kinopoisk.get_text()
movie_rating_dict["Рейтинг IMDb"] = movie_rating_imdb.get_text()
movie_rating_dict["Количество оценок IMDb"] = movie_rating_count_imdb.get_text()

In [33]:
movie_rating_dict

{'Рейтинг кинопоиска': '8.0',
 'Количество оценок кинопоиска': '142 538 оценок',
 'Рейтинг IMDb': 'IMDb: 8.40',
 'Количество оценок IMDb': '938 581 оценка'}

# Preparing final dict with information about the film

In [34]:
movie_dict = {**movie_id_dict, **movie_name_dict, **movie_info_dict, **movie_rating_dict}
pp.pprint(movie_dict)

{'id': 339,
 'Бюджет': '$18 000 000',
 'Возраст': '12+',
 'Время': '115 мин. / 01:55',
 'Год производства': '1981',
 'Жанр': 'приключения, боевик, комедия',
 'Зрители': '88.1 млн , 6.4 млн , 4.2 млн , ...',
 'Количество оценок IMDb': '938 581 оценка',
 'Количество оценок кинопоиска': '142 538 оценок',
 'Композитор': 'Джон Уильямс',
 'Монтаж': 'Майкл Кан, Джордж Лукас',
 'Название': 'Индиана Джонс: В поисках утраченного ковчега (1981)',
 'Оператор': 'Дуглас Слоком',
 'Оргинальное название': 'Raiders of the Lost Ark',
 'Премьера в мире': '12 июня 1981, ...',
 'Продюсер': 'Ховард Дж. Казанян, Джордж Лукас, Фрэнк Маршалл, ...',
 'Режиссер': 'Стивен Спилберг',
 'Рейтинг IMDb': 'IMDb: 8.40',
 'Рейтинг MPAA': 'PGрекомендуется присутствие родителей',
 'Рейтинг кинопоиска': '8.0',
 'Релиз на DVD': '20 ноября 2003, «Premier Digital»',
 'Сборы в США': '$212 222 025',
 'Сборы в мире': '+ $141 766 000 = $353 988 025',
 'Слоган': '«Indiana Jones - the new hero from the creators of JAWS and STAR WA

# Parsing page with reviews

## Getting the code for the page with movie

In [3]:
# f = requests.get("https://www.kinopoisk.ru/film/339/reviews/")

## Getting the code from the downloaded page

In [35]:
path_reviews = "./data/indiana_reviews.html"
with open(path_reviews, 'r', encoding="utf-8") as f:
    indiana_reviews = f.read()

Passing the code into the `BeautifulSoup` constructor

In [36]:
soup_reviews = BeautifulSoup(indiana_reviews, "html.parser")

In [37]:
reviews_dict = {}

## Parsing movie's id

In [47]:
reviews_id_dict = {}

In [48]:
film_id = soup_reviews.find_all("link", attrs={"href": re.compile("https://www.kinopoisk.ru/film/")})[0]

In [49]:
film_id = film_id["href"].split("/")[-3]

In [51]:
reviews_id_dict["id"] = int(film_id)
reviews_id_dict

{'id': 339}

## Parsing reviews

In [53]:
reviews = soup_reviews.find_all("div", attrs={"class": "reviewItem userReview"})

In [54]:
reviews[0]

<div class="reviewItem userReview" data-id="3128202" id="_DRAGGABLE_3128202" style="width: 555px; position: relative; padding: 1px 0 40px 0">
<div id="3128202"></div>
<div class="response good" id="div_review_3128202" itemprop="reviews" itemscope="" itemtype="http://schema.org/Review">
<meta content="Знакомство с Индианой Джонс в 2022." itemprop="headline"/>
<div itemprop="author" itemscope="" itemtype="http://schema.org/Person">
<img alt="" data-lazy-src="https://avatars.mds.yandex.net/get-kino-vod-users-avatar/57335/63366919-43-695797.jpg/46x73_bw" itemprop="image" src="https://st.kp.yandex.net/images/spacer.gif" title="" width="46"/>
<div>
<p class="profile_name"><s></s><a data-popup-info="disabled" href="/user/63366919/" itemprop="name">komisar.jeber</a></p>
<ul class="actions">
<li class="first"><a href="/user/63366919/comments/">рецензии</a></li>
<li><a href="/user/63366919/votes/">оценки</a></li>
<li><a href="/user/63366919/friends/">друзья</a></li>
<li><a href="/user/63366919/m